# 1. Getting Started

multiTFA is a python package for accurate thermodynamic flux analysis. multiTFA is built on COBRApy to take advantage of COBRA model structure. For installation instructions please refer to **INSTALL.rst**.

## 1.1. Building a multiTFA model

multiTFA models can be readily constructed from COBRA model objects. To create a model, we require 

1. A COBRA model

2. Metabolite KEGG/SEED identifiers - This is required to match and retrieve thermodyanamic information against our database. Metabolite identifiers can be added to the multiTFA model later. For simplicity, we use the same nomneclature as `equilibrator-api`. 

3. Compartment specific pH and ionic strength information. To calculate Gibbs energy related to transmembrane transport, information about membrane potential is also required in the form of pandas Dataframes. 

Additionally, for accurate thermodynamic calculations, intracellular metabolite concentrations can be provided. 

**NOTE**: All the input information should be in S.I units, e.g: ionic strength, metabolite concentration in Moles.

To demonstrate the usage, we use *E. Coli* core model from `BIGG database`.


In [ ]:
from cobra import io
from multitfa.core import tmodel

# Load COBRA model
cobra_model = io.load_matlab_model('e_coli_core.mat')

# compartment specific properties
pH_I_dict = {
    "pH": {"c": 7.5, "e": 7, "p": 7},
    "I": {"c": 0.25, "e": 0, "p": 0},
}

# Membrane potentials
del_psi_dict = { 
"c": {"c":0,"e": 0, "p": 150}, 
"e": {"c": 0,"e":0, "p": 0}, 
"p": {"c": -150, "e": 0, "p":0}, 
} 

import pandas as pd
# Define membrane potential and compartment information in Dataframes
del_psi = pd.DataFrame.from_dict(data=del_psi_dict)
comp_info = pd.DataFrame.from_dict(data=pH_I_dict)

# List of reaction ids that you would like to exclude from thermodynamic analysis, here we exclude demand reactions
Excl = [
    rxn.id
    for rxn in cobra_model.reactions
    if rxn.id.startswith("EX_") or rxn.id.startswith("DM_")
] + ["BIOMASS_Ecoli_core_w_GAM", "O2t", "H2Ot"]


# Now build the multiTFA model from COBRA model
tfa_model = tmodel(
    cobra_model, Exclude_list=Excl, compartment_info=comp_info, membrane_potential=del_psi
)


In order to access the thermodynamic information from equilibrator api, we need to add a external database identifier against all the metabolites in tfa model. This can be done by populating the 'Kegg_id' property of the metabolites. If you do not know the identifier for some metabolites, it is okay, you can simply add `NA` against it. For simplicity, we use the notation similar to equilibrator api. For the example below in the *E. coli* core model, we use bigg identifiers. You can follow equilibrator api notation for other database identifiers, for example, if you are using `KEGG` identifier, simply use `kegg:kegg identifier`.

In [ ]:
for met in tfa_model.metabolites: 
    kegg_id = 'bigg.metabolite:'+met.id[:-2] 
    met.Kegg_id = kegg_id

tfa_model.update()

## 1.2 Metabolites

Reactions and metabolites in multiTFA model can be accessed similar to a COBRA model. For demonstration, we will consider cytosolic `ATP` as our metabolite with `atp_c` identifier in tfa_model. We will access `ATP` and print it's standard Gibbs energy of formation and Also, change its intracellular concentration lower and upper bounds to '2e-3' and '5e-2' M respectively from it's default bounds. These changes will reflect in solver variables and constraints.

In [ ]:
atp = tfa_model.metabolites.get_by_id('atp_c')
print("Standard Gibbs energy of formation of ATP is {} KJ/mol".format(atp.delG_f))
print("initial concentrations")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))
atp.concentration_min = 2e-3
atp.concentration_max = 5e-2
print("Concentrations after modifying")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))

You can always change the Gibbs energy of formation value of a metabolite by resetting the `delG_f` property of the metabolite. Also, you can access additional information like, major pseudoisomer at given compartment conditions, equilibrator accession of the metabolite.

## 1.3 Reactions

Reactions in multiTFA model are of type cobra.DictList. Similar to COBRA model, reactions can be accessed/retrieved by its index or by it's name and properties/attributes can be accessed. Let us consider ATP synthase raction in our model by 'ATPS4rpp' and access it's attributes.


In [ ]:
atps = tfa_model.reactions.get_by_id('ATPS4r')
print(atps.reaction)

We can access the Gibbs energy of transport and transformed Gibbs energy of the reaction

In [ ]:
print("Gibbs energy of transport is {} KJ/mol".format(atps.delG_transport))
print("Transform Gibbs energy of is {} KJ/mol".format(atps.delG_prime))

You can always reset the transport and transformed Gibbs energy of reactions. You can also access the stoichiometric matrix of the reaction, all the thermodynamic constraints associated with the reaction.

# 2. Solving a multiTFA model

Once you add database identifiers, Compartment information and membrane potential data to the model, you're ready to poupulate the model with thermodynamic constraints. multiTFA offers two different ways to solve the tMFA problem (Please refer to manuscript), 

1. Univariate type, where each group/component is allowed to vary in 95% confidence interval from respective mean. This is formulated as a mixed integer linear programming problem (MILP)

2. Multivariate, where group/component are drawn from multivariate normal distribution subjected to the associated linear constraints. This is a mixed integer quadratic constraint program (MIQCP). In general, MIQCP are computationally expensive.

## 2.1. populating the thermodynamic constraints

multiTFA model solver is a `optlang` model. By adding the thermodynamic constraints to the model, you're adding new variables and constraints to the `optlang` model. `optlang` currently, doesn't support quadratic constraints. So, we currently support `Cplex/Gurobi` solvers for MIQC problems. You have to access the MIQCP through the model's `gurobi_interface/cplex_interface`. These interfaces are accessible depending on what solver you are using to solve the optlang model. For example, if you are using `Cplex` as your default solver, `cplex_interface` will be available to solve the MIQCP. You can always change the solver using,



In [ ]:
# Changing solver to cplex
tfa_model.solver = 'cplex'

populating the model is easy, you simply use the `update` function. Now all the thermodynamic constraints are added to the model. Now the model contains thermodynamic constraints along with the mass balance constraints. To see if we have added all the constraints properly, you can check the number of constraints 

In [ ]:
print("Number of constraints before update {}".format(len(cobra_model.constraints)))
tfa_model.update()
print("Number of constraints after update {}".format(len(tfa_model.constraints)))
print(len(tfa_model.metabolites) + 6*(len(tfa_model.reactions) - len(tfa_model.Exclude_reactions))) # To check if we have added correct number of constraints

## 2.2. Solving the model

Once we are satisfied with the constraints, we can proceed to solve the problem. By default, `tfa_model` attempts to return the solution to MIQC problem. If `Gurobi/Cplex` is not available, then it proceeds to solve the MILP. The solution structure is a `pd.Dataframe` and you can access Fluxes, Gibbs energies of reactions and metabolite concentrations.

In [ ]:
solution =  tfa_model.optimize(solve_method = 'mip') 
print(solution)

You can always update the properties and they are reflected in the variable bounds. For example, you can change concentration bounds for metabolites and they are automatically changed for both MILP and MIQCP.

As mentioned before, MIQCP are not `optlang` models and are individual `Gurobi/Cplex` models. You can access/modify their respective interfaces according to their api.

# 3. Thermodynamic variability analysis (TVA)

There exists multiple flux states that define the same optimum. TVA predicts the ranges of variables such as metabolic fluxes, Gibbs energy of reactions and metabolite concentrations.

As described previously, you can solve the TVA problem on either the MILP or MIQCP. Although, MILP is faster compared to solving MIQCP, MIQCP is thermodynamically consistent as they drawn from multivariate distribution. 

There are three different functions for running TVA in multiTFA. One for the MILP, the remaining two for MIQCP, depending on the solver you use (either Gurobi or Cplex).

By default, TVA is performed on all the variables in a model unless `variable_list` is specified. Please note, `variable_list` is a `list` of `str`. For example reactons id's or names of Gibbs energy variables etc. 


In [ ]:
from multitfa import analysis
rxn_vars = [rxn.id for rxn in tfa_model.reactions if rxn.id not in tfa_model.Exclude_reactions] # Flux variable names
dg_vars = [rxn.delG_forward.name for rxn in tfa_model.reactions if rxn.id not in tfa_model.Exclude_reactions] # Gibbs energy variable names

ranges = analysis.variability(tfa_model, variable_list=rxn_vars + dg_vars) # TVA using MILP

ranges_miqc = analysis.variability_legacy_cplex(tfa_model,variable_list=rxn_vars + dg_vars) # TVA using MIQCP with cplex

ranges_miqc = analysis.variability_legacy_gurobi(tfa_model,variable_list=rxn_vars + dg_vars) # TVA using MIQCP with Gurobi

Setting the parameter `fraction_of_optim=0.9` would give variable ranges at 90% optimality of specified variable optimality. The variable can be specified by Setting `biomass_rxn` parameter.  

analysis.variability_legacy_cplex(tfa_model,variable_list=vars_analysis, biomass_rxn='BIOMASS_Ec_iJO1366_core_53p95M', fraction_of_optim=0.9)

Please note, runnig TVA on genome scale model is computationally expensive, especially using MIQC problems. We have observed that relaxing the `Gap` parameter helped achieving faster run times. 

# 4. Debugging

It is often observed in genome scale models that adding thermodynamic constraints make the model infeasible. This might be due to variety of reasons, like ill-formatted  covariance between groups/components, lumping of reactions etc. It is always not easy to find the exact cause of the problem. But, if you are using Gurobi/Cplex, you can find the constraints that render the model infeasible. 

Please note that these methods give you random set of constraints that represent infeasibility. Also, a model can contain multiple infeasible sets. While modifying/removing these constraints, it is advisable to look for biological meaning.


In [ ]:
# Using gurobi
t_model.solver.problem.computeIIS() # model.solver.problem represents the solver interface model
for cons in t_model.solver.problem.getConstrs(): 
    if cons.IISConstr: 
        print(cons) 

# If using Cplex
t_model.solver.problem.conflict.refine(t_model.solver.problem.conflict.all_constraints())

